In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def split_real_im(c): 
    c = complex(c)
    return pd.Series([c.real,c.imag])

In [3]:
# import input and target data 
all_input = pd.read_csv("../Samples_RTS24/oT_Result_NN_Input_RTS24.csv")
all_output = pd.read_csv("../Samples_RTS24/oT_Result_NN_Output_RTS24.csv")

In [4]:
timestamps = np.unique(all_input.LoadLevel)
variables = np.unique(all_input.Variable)
executions = np.unique(all_input.Execution)
periods = np.unique(all_input.Period)
scenarios = np.unique(all_input.Scenario)

In [8]:
executions

array(['Network_Full_Generation_Full',
       'Network_Line_Out_N_101_N_102_cac1',
       'Network_Line_Out_N_101_N_103_cac1',
       'Network_Line_Out_N_101_N_105_cac1',
       'Network_Line_Out_N_102_N_104_cac1',
       'Network_Line_Out_N_102_N_106_cac1',
       'Network_Line_Out_N_103_N_109_cac1',
       'Network_Line_Out_N_103_N_124_cac1',
       'Network_Line_Out_N_104_N_109_cac1',
       'Network_Line_Out_N_105_N_110_cac1',
       'Network_Line_Out_N_106_N_108_cac1',
       'Network_Line_Out_N_106_N_110_cac1',
       'Network_Line_Out_N_107_N_108_cac1',
       'Network_Line_Out_N_108_N_109_cac1',
       'Network_Line_Out_N_108_N_110_cac1',
       'Network_Line_Out_N_109_N_111_cac1',
       'Network_Line_Out_N_109_N_112_cac1',
       'Network_Line_Out_N_110_N_111_cac1',
       'Network_Line_Out_N_110_N_112_cac1',
       'Network_Line_Out_N_111_N_113_cac1',
       'Network_Line_Out_N_111_N_114_cac1',
       'Network_Line_Out_N_112_N_113_cac1',
       'Network_Line_Out_N_112_N_123_

In [7]:
#Select a certain scenario and execution

#exe = executions[5]
per = periods[0]
sc = scenarios[0]
for exe in executions[5:]:

    variables_to_keep = variables[4:]
    variables_to_keep_o = np.sort(np.unique(all_output.Variable))[[0,1,3,4]]

    #Filter the main frame on desired parameters: 
    f_i = (all_input.LoadLevel.isin(timestamps))& (all_input.Execution == exe ) & (all_input.Period == per )& (all_input.Scenario == sc)& (all_input.Variable.isin(variables_to_keep))
    input_f = pd.DataFrame(all_input[f_i])

    f_o = (all_output.LoadLevel.isin(timestamps))& (all_output.Execution == exe ) & (all_output.Period == per )& (all_output.Scenario == sc)& (all_output.Variable.isin(variables_to_keep_o))
    output_f = pd.DataFrame(all_output[f_o])

    #And then add to new frame the two parts of the complex numbers: 
    input_f[['Value_R', 'Value_I']] = input_f["Value"].apply(split_real_im)

    #Pivot 
    input_f_p = input_f.pivot(index = "LoadLevel",columns=["Variable"],values = ["Value_R","Value_I"])
    output_f_p = output_f.pivot(index = "LoadLevel",columns=["Variable"],values = ["Value"])

    #Write to separate csv file
    input_f_p.to_csv(f"Data/RTS_24/input_f_{sc}_{exe}_{per}.csv")
    output_f_p.to_csv(f"Data/RTS_24/output_f_{sc}_{exe}_{per}.csv")